# Explore population density in the bay area

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Modules and Packages



In [3]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

In [12]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    costs = {}
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        costs['total_cost'] = total_cost
        costs['minutes'] = round(total_cost / 60.0,1)
    
    return(costs)
    

In [5]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [6]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    return(pd.DataFrame(cursor.fetchall(), columns =['zip', 'population']))
        

In [7]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)


In [8]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [9]:
cursor = connection.cursor()

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

session = driver.session(database="neo4j")

In [14]:
my_neo4j_shortest_path('depart Downtown Berkeley','arrive Antioch')

{'total_cost': 3659, 'minutes': 61.0}

# Find all zip codes and population, within 1, 3, 5, and 10 miles of the Downtown Berkeley station

To start, we'll look at all of the zip codes and population of those zip codes that we can service from the Downtown Berkley station. We'll use these panda's dataframes to compare to other stations to ensure that we're reaching net new cusotmers.

In [6]:
downtown_berkley_1_mile = my_station_get_zips('Downtown Berkeley', 1)
downtown_berkley_3_mile = my_station_get_zips('Downtown Berkeley', 3)
downtown_berkley_5_mile = my_station_get_zips('Downtown Berkeley', 5)
downtown_berkley_10_mile = my_station_get_zips('Downtown Berkeley', 10)

downtown_berkley_1_mile

,zip,population
0,94702,17092
1,94703,21937
2,94704,29190
3,94709,11740
4,94720,2971


# Get list of all other stations excluding 'Downtown Berkley'

In [18]:
def get_stations():
    connection.rollback()
    
    query = "select station from stations where station <> 'Downtown Berkley'"

    cursor.execute(query)

    stations = []
    temp_stations = cursor.fetchall()
    for station in temp_stations:
        stations.append(station[0])
    
    return stations

# Fin all zip codes and population within a distence of every station

At least 75% of the population must be in zip codes that are not within the same distence of the Downtown Berkeley station

In [19]:
def find_net_new_customers(distance):
    """finds all stations where 50% of the population within the distence would be net new"""
    stations = get_stations()
    
    berkeley_zips = my_station_get_zips('Downtown Berkeley', distance)
    berkeley_pop = berkeley_zips['population'].sum()
    
    new_stations = []
    
    for station in stations:
        station_zips = my_station_get_zips(station, distance)
        
        new_pop = station_zips['population'].sum()
        overlap_zips = pd.DataFrame(berkeley_zips.merge(station_zips, right_on='zip', left_on='zip',how='inner'))
        
        overlap_zip_pop = overlap_zips['population_x'].sum()
        
        percent_new = ( new_pop / overlap_zip_pop ) if overlap_zip_pop != 0 else 0
        
        if percent_new < .75:
            new_stations.append(station)
    
    return new_stations

In [22]:
stations_outside_distance = {}
for c in find_net_new_customers(8):
    stations_outside_distance[c] = my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes']

stations_outside_distance

{'Antioch': 61.0,
 'Berryessa': 70.0,
 'Concord': 35.0,
 'Dublin': 49.9,
 'Fremont': 50.0,
 'Millbrae': 60.0,
 'Milpitas': 65.0,
 'North Concord': 38.0,
 'Pittsburg': 44.0,
 'Pittsburg Center': 54.0,
 'SFO': 57.8,
 'San Bruno': 53.0,
 'South Hayward': 40.0,
 'Union City': 45.0,
 'Warm Springs': 56.0,
 'West Dublin': 46.9}

In [26]:
stations_outside_train_length = {}

shorest = stations_outside_distance[list(stations_outside_distance.keys())[0]]

for i in list(stations_outside_distance.keys()):
    if stations_outside_distance[i] < shorest:
        shorest = stations_outside_distance[i]

for c in get_stations():
    if my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes'] > shorest:
        stations_outside_train_length[c] = my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes']

In [27]:
stations_outside_train_length

{'Antioch': 61.0,
 'Balboa Park': 38.0,
 'Berryessa': 70.0,
 'Castro Valley': 36.9,
 'Colma': 46.0,
 'Daly City': 42.0,
 'Dublin': 49.9,
 'Fremont': 50.0,
 'Glen Park': 36.0,
 'Hayward': 36.0,
 'Millbrae': 60.0,
 'Milpitas': 65.0,
 'North Concord': 38.0,
 'Pittsburg': 44.0,
 'Pittsburg Center': 54.0,
 'SFO': 57.8,
 'San Bruno': 53.0,
 'South Hayward': 40.0,
 'South San Francisco': 49.0,
 'Union City': 45.0,
 'Warm Springs': 56.0,
 'West Dublin': 46.9}

In [ ]:
# look at customers who haven't made a sale yet
# find the zip codes where we have customers but they don't come into the store very often

In [34]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    c.zip as zip,
    count( distinct( c.customer_id ) ) as customers,
    sum(s.total_amount) as revenue
    
from customers c
    join sales s on c.customer_id = s.customer_id
    join stores st on c.closest_store_id = st.store_id
    
where st.city = 'Berkeley'

group by 1

order by 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,customers,revenue
0,94608,230,914136
1,94602,259,848292
2,94530,258,827292
3,94606,198,738876
4,94611,203,733620
...,...,...,...
139,94588,1,1836
140,94565,1,1776
141,94957,1,1752
142,94516,1,1680


In [35]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    c.distance as distance,
    count( distinct( c.customer_id ) ) as customers,
    sum(s.total_amount) as revenue
    
from customers c
    join sales s on c.customer_id = s.customer_id
    join stores st on c.closest_store_id = st.store_id
    
where st.city = 'Berkeley'

group by 1

order by 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,distance,customers,revenue
0,1,790,3128808
1,3,745,3027780
2,4,796,2877264
3,2,593,2324136
4,6,569,1729116
5,5,518,1677264
6,9,546,1450080
7,8,428,1331412
8,10,532,1293816
9,7,375,1138368
